In [ ]:
import yaml
config = '../inputs/config.yml'

data = yaml.safe_load(open(config))

print(config)
print(data)

In [ ]:
import core

config = '../inputs/config.yml'
connection = '../inputs/connection.yml'

schema = 'test'

configuration = core.read_configuration(config)
print(configuration)
core.create_local_folders_and_files(config)
core.create_database(config, connection)
core.create_extensions(config, connection)
core.create_users(config, connection)
core.create_schemas(config, connection)  # maybe use https://github.com/RazerM/pg_grant to persist the privilege granting?
core.create_policies(config, connection) # maybe use https://github.com/RazerM/pg_grant to persist the privilege granting?

In [ ]:
# Revert all above steps.

import yaml
import os
import shutil

from sqlalchemy import create_engine                # noqa: F401
from sqlalchemy import text, quoted_name            # noqa: F401
from sqlalchemy.exc import SQLAlchemyError          # noqa: F401
import sqlalchemy_utils as sal_utils                # noqa: F401


import utils.db_connect as db_connect
from core.read_configuration import read_configuration
from utils.random_password_generator import generate_password

config = '../inputs/config.yml'
connection = '../inputs/connection.yml'


# read the configuration
configuration = read_configuration(config)

# define db_name and define directory path as absolute path
db_name = configuration['db_name']
db_path = configuration['paths']['db_path']

# PostgreSQL connection information
conn_string = db_connect.get_db_connection(config, connection)

# Create the SQLAlchemy engine
engine = create_engine(conn_string)

print(configuration['users'])

for user in configuration['users']:
    print(user)
    drop_user = text(f"drop user {quoted_name(user, False)};")
    
    # Drop users
    with engine.connect() as conn:
        transaction = conn.begin()
        conn.execute(drop_user)
        transaction.commit()
    conn.close()

# PostgreSQL connection information
sal_utils.drop_database(engine.url)

# remove folder structure
shutil.rmtree(db_path)
